In [ ]:
#| default_exp synteny

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

In [ ]:
#| exporti
import time

import pandas as pd

from skbio.io import read as skbio_read
from skbio.metadata import IntervalMetadata
from skbio.sequence import DNA

from pygengraph.utils import bidict,inverseSequence

In [ ]:
#| export
def readTransMap(transMapFile,ATaccessionName='araport',similarityGroupColName='Orthogroup'):
    '''
    Read trandmap file which is pretty much a tsv file with first column containing orthogroup, or similarity group ID,
    and the rest of columns correspond to each accession with unique gene/interval IDs in each columns for the given orthogroup.
    If one accession has several genes in the given orthogroup, then they will be in a comma separated list.
    
    Parameters
    ==========
    transMapFile: path to the file to read
    
    ATaccessionName: Which accession to read genes from? Usually used to read reference gene IDs, but can be any accession.
                     If it is given a `bidict` object of two way relationship between similarity group IDs and this accession
                     gene IDs is returned. Otherwise, a Pandas DataFrame read from the file is returned.
    
    similarityGroupColName: Column name with similarity group IDs (index column).
    
    '''
    
    # reading and preparing transmap in pandas format
    transMap = pd.read_csv(transMapFile,delimiter='\t')
    transMap.fillna('',inplace=True)
    transMap.rename(columns={similarityGroupColName:'orthogroup'},inplace=True)
    transMap['orthogroup'] = transMap.orthogroup.str.rstrip(':')
    transMap.set_index('orthogroup',inplace=True)
    if ATaccessionName is not None:
        ATTransMap = transMap[ATaccessionName]
        return bidict({og:ATTransMap[og].split(', ') for og in ATTransMap.index})
    else:
        return transMap

In [ ]:
#| export
def generateOrder(files, priorityAccession='TIAR10'):
    '''
    Genrate a list of files and float a file which contains specific priority accession name (or any given string)
    on top of the list. 
    
    It is not used any more, but left if will be needed in the future.
    '''
    idxList = list(range(len(files)))
    if priorityAccession is not None:
        ind = [idx for idx, file in enumerate(files) if priorityAccession in file][0]
        del idxList[ind]
        idxList = [ind] + idxList
    return idxList

In [ ]:
#| export
def getIDs(iterator):
    idList = []
    for interval in iterator:
        idList.append(interval.metadata['ID'])
    
    return idList

In [ ]:
#| export
def readGFF(gffFile: str):
    '''
    Function which reads GFF3 file into a dict structure. For large GFF files random search is extremely slow and is impossible to use.
    This greatly speed up the process (from hours to seconds).
    '''
    annotation = skbio_read(gffFile,format='gff3')
    annDict = {}

    for seqid,intList in annotation:
        annDict.setdefault(seqid,IntervalMetadata(None)).merge(intList)
        
    return annDict

In [ ]:
#| export
def addPangenomePositions(pangenomeFiles):
    if pangenomeFiles is None:
        return {}
    
    numFiles = len(pangenomeFiles)
    
    for pangenomeFile in pangenomeFiles:
        pangenAnnDict = readGFF(pangenomeFile)
        print(f'Pangenome annotation file {pangenomeFile} read.')

        pangenGeneDict = {}
        numSeqs = len(pangenAnnDict)

        for si,[seqID,pangenAnn] in enumerate(pangenAnnDict.items()):

            print(f'Processing pangenome sequence {si+1}/{numSeqs}')
            chromosome = ''.join(seqID.split('_')[-2:]) # This specifies how to extract molecule name from pangenome sequence ID. If it changes, should also be changed.
            pangenChrDict = pangenGeneDict.setdefault(chromosome,{})
            
            for gi,gene in enumerate(pangenAnn.query(metadata={'type': 'gene'})):
                print(f'\rProcessing gene number {gi}', end = '')
                pangenChrDict[gene.metadata['ID']] = gene.bounds[0]

            print()

    return pangenGeneDict

In [ ]:
#| export
def processAccessions(annotationFiles, ATmap = None, pangenomeDict = None,
                      similarityIDKey = None, similarityIDAssignment = None, 
                      pangenomeFiles=None, sequenceFilesDict=None, 
                      seqidJoinSym = '_', ATsplitSym = ','):
    '''
    This function process only custom annotation for a group of accession with similarity IDs. It also creates separate ATmap and 
    returns the last number of unmatchedID for consistency. There is a separate function for processing reference annotation.
    
    `sequenceFilesDict` can either be None or dict with accession IDs as keys and paths to FASTA files with sequences of annotated elements as values.
    
    `pangenomeFiles`: list[str] or None. If list of strings is provided, then its length should be the same length as `annotationFiles` and should be
                    in the same order as in `annotationFiles`, i.e. `pangenomeFiles[i]` should correspond to `annotationFiles[i]`
    
    `similarityIDAssignment`: str. Can be either 'gene' or 'mRNA'. If it is not any of these an error will be raised.
    '''
    
    genes = {}
    unmatchedID = 0
    anngrTosimgr = {}
    def generateUID(anngr):
        nonlocal unmatchedID
        nonlocal anngrTosimgr
        if anngr in anngrTosimgr:
            return anngrTosimgr[anngr]
        else:
            UID = f'UID{unmatchedID:010d}'
            anngrTosimgr[anngr] = UID
            unmatchedID += 1
            return UID
    # id generator f'UID{q:010d}'
    
    sequenceDict = None
    
    if ATmap is None:
        _ATmap = bidict()
    chromosomes = set()
    
    if pangenomeDict is None:
        pangenomeDict = addPangenomePositions(pangenomeFiles)
    
    overalltime = time.time()
    
    for fileID,annotationFile in enumerate(annotationFiles):
        filetime = time.time()
        
        print(f'Processing file {annotationFile}...')
        
        annotationDict = readGFF(annotationFile)
        print(f'Annotation file {annotationFile} read.')

        accessionID = None
        changeAccID = True
        
        for seqID,annotation in sorted(annotationDict.items()):
            
            seqtime = time.time()
            
            proposedAccID,chromosome = seqID.split(seqidJoinSym)
            
            chromosomes.add(chromosome)
            
            if accessionID != proposedAccID:
                accessionID = proposedAccID
                changeAccID = True
            
            if changeAccID:
                sequenceDict = None
                if sequenceFilesDict is not None and accessionID in sequenceFilesDict:
                    sequenceDict = {}
                    sequenceGen = skbio_read(sequenceFileDict[accessionID],format='fasta')
                    for seq in sequenceGen:
                        sequenceDict[seq.metadata['id']] = bytearray(seq.values).decode()
                changeAccID = False

            geneInts = list(annotation.query(metadata={'type':'gene'}))
            genenum = len(geneInts)

            for gi,gene in enumerate(geneInts):
                geneID = gene.metadata['ID']
                
                if gene.metadata['source'] == 'Consensus_tair10':
                    orthogroup = geneID
                    atNameList = geneID
                else:
                    if similarityIDAssignment=='gene':
                        orthogroup = gene.metadata[similarityIDKey]
                    elif similarityIDAssignment=='mRNA':
                        mrna = list(annotation.query(metadata={'type': 'mRNA', 'Parent': geneID}))
                        # FIX! If no mRNA, do not record gene?
                        if len(mrna)>0:
                            orthogroup = mrna[0].metadata.get('simgr',generateUID(geneID))
                        else:
                            orthogroup = generateUID(geneID)
                    elif ATmap is not None:
                        orthogroup = ATmap.inverse.get(geneID,generateUID(geneID))
                    else:
                        raise ValueError(f'`similarityIDAssignment` supports only "gene" or "mRNA", but {similarityIDAssignment} was given and ATmap is not available.')

                    if orthogroup is None:
                        continue

                
                if ATmap is None:
                    
                        if 'AT' in gene.metadata:
                            atNameList = gene.metadata['AT'].split(ATsplitSym)
                        elif 'Name' in gene.metadata:
                            atNameList = gene.metadata['Name'].split(ATsplitSym)
                        else:
                            atNameList = ['']
                
                forward = gene.metadata['strand']=='+'
                start,end = gene.bounds[0]
                
                pStart,pEnd = pangenomeDict.get(chromosome,{}).get(geneID,[-2,-1])
                
                if sequenceDict is not None:
                    geneSeq = sequenceDict[seqID][start:end+1]
                else:
                    geneSeq = ''
                
                overlaps = getIDs(annotation.query(bounds=[(start,end)],metadata={'type':'gene'}))
                genes.setdefault(accessionID,[]).append([geneID,orthogroup,seqID,accessionID,chromosome,forward,start+1,end,pStart+1,pEnd,geneSeq,overlaps])
                if ATmap is None:
                    if len(atNameList[0])>0:
                        _ATmap.add(orthogroup,atNameList)
        print(f'Processing file {annotationFile} finished in {time.time() - filetime} seconds.')
    
    print('Converting gene tables into pandas DataFrames.')
    
    for accID in genes.keys():
        genes[accID] = pd.DataFrame(genes[accID],columns=['geneID','orthogroup','sequenceID','accessionID','chromosome','forward','start','end','pstart','pend','geneSeq','overlapGenes'])
        genes[accID].sort_values(by=['chromosome','start'],inplace=True)

    print(f'Reading annotation files finished in {time.time() - overalltime} seconds')
    
    if ATmap is None:
        return genes,chromosomes,_ATmap,pangenomeDict,sequenceDict
    else:
        return genes,chromosomes,pangenomeDict,sequenceDict

In [ ]:
#| export
def recordSegment(name,segmentIDs,segmentIDToNumDict,sequence=None,gfaFile=None,segmentData=None):
    segmentIDs.append(name)
    
    segmentIDToNumDict[name] = len(segmentIDs)-1
    segID = len(segmentIDs)
    
    if segmentData is not None and sequence is not None:
        segmentData.append(sequence)
    
    if gfaFile is not None:
        if sequence is not None:
            gfaFile.write(f'S\t{segID}\t{sequence}\n')        
        else:
            gfaFile.write(f'S\t{segID}\t{name}\n')
    return segID

In [ ]:
#| export
def recordAnnotation(nodeID,accessionID,sequenceID,chrID,start,end,og,atList,sequence,nodesMetadata,pstart=-1,pend=-1):

    if len(nodesMetadata)==nodeID-1:
        nodesMetadata.append({})

    geneLen = 1
    if len(sequence)>0:
        geneLen = len(sequence)

    nodesMetadata[nodeID-1].setdefault(accessionID, {}).setdefault('posisions',[]).append({'gen':{'chr':chrID, 'genomePosition':[start,end]}})
    
    if pstart != -1 and pend != -1:
        nodesMetadata[nodeID-1][accessionID]['positions'][-1]['pangen'] = {'chr':chrID, 'genomePosition':[pstart,pend]}
    
    nodesMetadata[nodeID-1][accessionID].setdefault('annotation',{}).update({og: [(0,geneLen-1)]})
    nodesMetadata[nodeID-1][accessionID]['annotation'].update({at:[(0,geneLen-1)] for at in atList})

In [ ]:
#| export
def recordAltChr(nodeID,accessionID,chrID,start,end,nodesMetadata):
    nodesMetadata[nodeID-1][accessionID].setdefault('altChrGenPos',[]).append({'chr':chrID, 'genomePosition':[start,end]})

In [ ]:
#| export
def addLink(links,prevPathSegment,name,forward):
    '''
    `links`: mutable
    `prevPathSegment`: mutable
    '''
    if prevPathSegment is not None:
        links[prevPathSegment].add(f'{name}\t{"+" if forward else "-"}')
    return f'{name}\t{"+" if forward else "-"}'

In [ ]:
#| export
def generatePathsLinks(genesAll,ATmap,accessionID,
                       sequences,OGList,segmentIDs,
                       nodesMetadata,
                       segmentIDToNumDict,links,usCounter,
                       chromosomeID = None,
                       doUS=True,segmentData=None,gfaFile=None):
    '''
    This function takes a list of genes in specific format (`genesAll`) and some extra data and pretty much generates a graph 
    (gene graph from annotations).
    
    `gfaFile`: file handle to write segments to GFA file
    `OGList`: mutable
    `links`: mutable
    `usCounter`: mutable
    
    '''
    print('Converting gene table into graph...')
    overalltime = time.time()
    
    genes = genesAll.loc[genesAll.chromosome == chromosomeID]
    
    path = []
    cigar = []
    prevEnd = 0
    prevPathSegment = None
    curSeqID = ''
    numGenes = len(genes)
    for gi,[generow,gene] in enumerate(genes.iterrows()):
        print(f'\rProcessing genes {gi+1}/{numGenes}', end='')
        og = gene.orthogroup
        
        geneSeqID = gene.sequenceID
        if curSeqID != geneSeqID:
            curSeqID = geneSeqID
        
        atStr = ATmap.get(og,[''])
        if len(atStr[0])>0:
            atList = atStr
        else:
            atList = []
        
        geneChr = gene.chromosome
        geneStart = gene.start
        geneEnd = gene.end
        genePStart = gene.pstart
        genePEnd = gene.pend
        geneForward = gene.forward
        
        if sequences is not None:
            if geneForward:
                geneSeq = sequences[geneSeqID][geneStart-1:geneEnd]
            else:
                geneSeq = inverseSequence(sequences[geneSeqID][geneStart-1:geneEnd])
        else:
            geneSeq = ''
        
        if doUS:
        
            if sequences is not None:
                usSeq = sequences[geneSeqID][prevEnd-1:geneStart]
            else:
                usSeq = ''

            if len(usSeq)>0:
                isUS = True
                us = f'US{usCounter:07d}'
                usCounter += 1
            else:
                isUS = False

            if isUS:
                usID = recordSegment(us,segmentIDs,segmentIDToNumDict,usSeq,gfaFile=gfaFile,segmentData=segmentData)
                recordAnnotation(usID,accessionID,geneSeqID,geneChr,prevEnd,geneStart-1,us,[],usSeq,nodesMetadata)
        
        if og not in OGList:
            ogID = recordSegment(og,segmentIDs,segmentIDToNumDict,geneSeq,gfaFile=gfaFile,segmentData=segmentData)
            OGList.append(og)
        else:
            ogID = segmentIDs.index(og)+1
        
        recordAnnotation(ogID,accessionID,geneSeqID,geneChr,geneStart,geneEnd,og,atList,geneSeq,nodesMetadata)
        if chromosomeID is not None:
            altPos = genesAll.loc[(genesAll.orthogroup == og) & (genesAll.chromosome != chromosomeID)]
            for altrow,altChrOG in altPos.iterrows():
                recordAltChr(ogID,accessionID,altChrOG.chromosome,altChrOG.start,altChrOG.end,nodesMetadata)
        
        pathAdd = [f'{ogID}{"+" if geneForward else "-"}']
        if doUS and isUS:
            pathAdd.insert(0,f'{usID}+')
            
        path.extend(pathAdd)
        
        if len(cigar)>0 and doUS and isUS:
            cigar.extend(['0M','0M']) # with previous block and between two current blocks
        else:
            cigar.append('0M') # only between current blocks or between previous and current gene
                               # without unrelated sequence (intergenic) block.
        
        if doUS and isUS:
            prevPathSegment = addLink(links,prevPathSegment,usID,True)
            links[prevPathSegment] = set()
        
        prevPathSegment = addLink(links,prevPathSegment,ogID,geneForward)
        if prevPathSegment not in links:
            links[prevPathSegment] = set()
        
        prevEnd = geneEnd+1
    
    if doUS:
        
        if sequences is not None:
            usSeq = sequences[curSeqID][prevEnd-1:]
        else:
            usSeq = ''

        if len(usSeq)>0:
            us = f'US{usCounter:07d}'
            usID = recordSegment(us,segmentIDs,segmentIDToNumDict,usSeq,gfaFile=gfaFile,segmentData=segmentData)
            recordAnnotation(usID,accessionID,geneSeqID,prevEnd,len(sequences[curSeqID]),us,[],nodesMetadata)
            usCounter += 1
            path.append(f'{usID}+')
            cigar.append('0M')
            prevPathSegment = addLink(links,prevPathSegment,usID,True)
    
    print(f'\nProcessing gene table finished in {time.time() - overalltime}')
    
    return path,cigar,usCounter

In [ ]:
#| export
def writeLinks(gfaFile,links,doCigars=True):
    for linkLeft,linksRight in links.items():
        for linkRight in linksRight:
            if doCigars:
                gfaFile.write(f'L\t{linkLeft}\t{linkRight}\t0M\n')
            else:
                gfaFile.write(f'L\t{linkLeft}\t{linkRight}\t*\n')
                
def writePath(gfaFile,AccessionID,path,cigar,doCigars):   
    if doCigars:
        cigarString = ",".join(cigar)
    else:
        cigarString = "*"
        
    gfaFile.write(f'P\t{AccessionID}\t{",".join(path)}\t{cigarString}\n')
    
def writeSegmentIDs(path,segmentIDs):
    with open(path,'w') as jsf:
        json.dump(segmentIDs,jsf)
        
def readSegmentIDs(path):
    with open(path,'r') as jsf:
        return json.load(jsf)